In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib as mpl
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.cross_validation import PredefinedSplit
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import sys
sys.path.append('.')
import utils

/home/guoshuai/.local/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# 单个lr
import  FeatureExact as feat

# load datasets and preprocessing
data_file = './data.csv'
data = feat.feature_select(data_file)
X_train, X_val, X_test, y_train, y_val, y_test = feat.split_dataset(data)
feat.test_lr_model(X_train, X_val, X_test, y_train, y_val, y_test)

[0]	train-auc:0.644228	test-auc:0.628106
[1]	train-auc:0.71091	test-auc:0.71108
[2]	train-auc:0.711962	test-auc:0.72494
[3]	train-auc:0.729051	test-auc:0.72989
[4]	train-auc:0.741489	test-auc:0.725341
[5]	train-auc:0.746052	test-auc:0.734132
[6]	train-auc:0.753162	test-auc:0.747373
[7]	train-auc:0.753036	test-auc:0.748983
[8]	train-auc:0.754054	test-auc:0.74432
[9]	train-auc:0.754275	test-auc:0.744218


/home/guoshuai/.local/lib/python3.5/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


lr-model validation result: 

percision: 0.0412 recall: 0.3360 f1: 0.0733 auc: 0.7494 ks: 0.3756 confusion_matrix: 
54165  3843   
326    165    

percision: 0.0345 recall: 0.2791 f1: 0.0614 auc: 0.7687 ks: 0.4148 confusion_matrix: 
9519   672    
62     24     

percision: 0.0353 recall: 0.3023 f1: 0.0633 auc: 0.7615 ks: 0.4309 confusion_matrix: 
9482   710    
60     26     


In [3]:
# 十折
import  FeatureExact as feat
data_file = './data.csv'
data = feat.feature_select(data_file)
data_y = data.pop('label').as_matrix()
data_X = data.as_matrix()

train_auc, train_ks = [], []
val_auc, val_ks = [], []
test_auc, test_ks = [], []
i = 0
lr = LogisticRegression(penalty='l2', C = 1, class_weight={1:120})

for X_train, X_val, X_test, y_train, y_val, y_test in utils.kfold(data_X, data_y, num_fold = 10):
    print(i)
    i = i + 1
    min_max_scaler = MinMaxScaler()
    min_max_scaler.fit(X_train)
    X_train = min_max_scaler.transform(X_train)
    X_val = min_max_scaler.transform(X_val)
    X_test = min_max_scaler.transform(X_test)

    lr.fit(X_train, y_train)
    # validate the  LR model
    y_train_pred_prob = lr.predict_proba(X_train)[:,1]
    y_val_pred_prob = lr.predict_proba(X_val)[:,1]
    y_test_pred_prob = lr.predict_proba(X_test)[:,1]

    temp_auc, temp_ks = utils.model_key_performance(y_train_pred_prob, y_train)
    train_auc.append(temp_auc)
    train_ks.append(temp_ks)
    temp_auc, temp_ks = utils.model_key_performance(y_val_pred_prob, y_val)
    val_auc.append(temp_auc)
    val_ks.append(temp_ks)
    temp_auc, temp_ks = utils.model_key_performance(y_test_pred_prob, y_test)
    test_auc.append(temp_auc)
    test_ks.append(temp_ks)

print(np.mean(train_auc), np.mean(train_ks))
print(np.mean(val_auc), np.mean(val_ks))
print(np.mean(test_auc), np.mean(test_ks))

[0]	train-auc:0.644228	test-auc:0.628106
[1]	train-auc:0.71091	test-auc:0.71108
[2]	train-auc:0.711962	test-auc:0.72494
[3]	train-auc:0.729051	test-auc:0.72989
[4]	train-auc:0.741489	test-auc:0.725341
[5]	train-auc:0.746052	test-auc:0.734132
[6]	train-auc:0.753162	test-auc:0.747373
[7]	train-auc:0.753036	test-auc:0.748983
[8]	train-auc:0.754054	test-auc:0.74432
[9]	train-auc:0.754275	test-auc:0.744218
0


/home/guoshuai/.local/lib/python3.5/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


1
2
3
4
5
6
7
8
9
0.760335648714 0.399986990678
0.724872642239 0.371585884354
0.746214190568 0.409368307747
